In [2]:
%pylab widget
import sys


sys.path.insert(0, '../')
sys.path.insert(0, '../../sidpy/')
import sidpy
from sidpy.io.interface_utils import open_file_dialog
print('sidpy version: ', sidpy.__version__)

from SciFiReaders import DM3Reader
import SciFiReaders
print('SciFiReaders version: ', SciFiReaders.__version__)

import pycroscopy as px
%load_ext autoreload

%autoreload 2

import pycroscopy.image

## Load Image

In [185]:
dialog = open_file_dialog()
dialog

FileChooser(path='.', filename='', title='HTML(value='', layout=Layout(display='none'))', show_hidden='False',…

In [91]:
dm3_reader = DM3Reader(dialog.selected)
dataset = dm3_reader.read()
pixel_size = 16/512*0.97
dataset.set_dimension(0, sidpy.Dimension(np.arange(dataset.shape[0])*pixel_size,
                                              name='x', units='nm', dimension_type='SPATIAL',
                                              quantity='length'))
dataset.set_dimension(1, sidpy.Dimension(np.arange(dataset.shape[1])*pixel_size,
                                              name='y', units='nm', dimension_type='SPATIAL',
                                              quantity='length'))

dataset.plot()
from matplotlib.widgets import RectangleSelector
# drawtype is 'box' or 'line' or 'none'
selector = RectangleSelector(plt.gca(), None ,drawtype='box', useblit=True,
                                       button=[1, 3],  # disable middle button
                                       minspanx=5, minspany=5,
                                       spancoords='pixels',
                                       interactive=True)

In [92]:
selection = px.image.crop_image(dataset, selector.corners)
selection.plot()

## Clean Image

In [93]:
# ----Input --------------
resolution = 0.05  # in nm
# ------------------------

lr_dset = pycroscopy.image.decon_lr(dataset-dataset.min(), resolution=resolution)

lr_dset[lr_dset>12] = 12
lr_dset.plot()

In [94]:
svd_dset = pycroscopy.image.clean_svd(dataset, source_size=3)
svd_dset.plot()

## Find Atoms
### Blob Finder with Pixel Accuracy

In [122]:
# ----Input ---------
atom_size = 0.1  # units of image scale
threshold = 0.03
# -------------------
atoms = pycroscopy.image.find_atoms(lr_dset, atom_size=atom_size, threshold=threshold)
print(len(atoms))

plt.figure()
plt.imshow(lr_dset.T, vmax=9)
plt.scatter(atoms[:, 0], atoms[:,1], color='red')

7632


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Refine Atom Positions 
Refine atom positions  with pixel sub-pixel accuracy by fitting a (symmetric) Gaussian in peak

In [96]:
sym = pycroscopy.image.atom_refine(np.array(lr_dset-lr_dset.min()+1e-12), atoms, 2, max_int = 0, min_int = 2, max_dist = 2)
refined_atoms = np.array(sym['atoms'])

plt.figure()
plt.imshow(lr_dset.T, vmax=9)
plt.scatter(refined_atoms[:, 0]+0.5, refined_atoms[:,1]+0.5, color='red')

In [17]:
import scipy.spatial

In [105]:
vor = scipy.spatial.Voronoi(atoms[:,:2])


In [175]:
tetrahedra = scipy.spatial.Delaunay(atoms[:100,:2])
extent = [lr_dset.shape[0], lr_dset.shape[1]]
voronoi_vertices = []
voronoi_tetrahedrons = []
r_vv = []
for vertices in tetrahedra.vertices:
    voronoi, radius = circumsphere_center(atoms[vertices,:2])
    print(voronoi, radius)
    if (voronoi >= 0).all() and (extent - voronoi > 0).all() and radius > 0.01:
        voronoi_vertices.append(voronoi)
        voronoi_tetrahedrons.append(vertices)
        r_vv.append(radius - r_a)
#    return voronoi_vertices, voronoi_tetrahedrons, r_vv

voronoi_vertices

[-135.91212195 -402.00516215] [        nan 259.5364144]
[ 81.5        320.74324324] [nan nan]


/tmp/ipykernel_256/3976049663.py:53: RuntimeWarning: invalid value encountered in sqrt
  circum_radius = np.sqrt(circum_center**2-vertex_pos[0]**2)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [120]:
atoms[:,2] = 0.

polyhedra  = find_polyhedra(atoms, 4, extent=[lr_dset.x[-1],lr_dset.y[-1],1])

Strange atomic radius, are you sure you know what you are doing?


QhullError: QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull d Qz Qbb Q12 Qc Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 848036582  delaunay  Qz-infinity-point  Qbbound-last  Q12-allow-wide
  Qcoplanar-keep  Qtriangulate  _pre-merge  _zero-centrum  Qinterior-keep
  Pgood  _max-width 5.1e+02  Error-roundoff 1e-12  _one-merge 9.3e-12
  Visible-distance 6.2e-12  U-max-coplanar 6.2e-12  Width-outside 1.2e-11
  _wide-facet 3.7e-11  _maxoutside 1.2e-11

precision problems (corrected unless 'Q0' or an error)
   7629 zero divisors during gaussian elimination

The input to qhull appears to be less than 4 dimensional, or a
computation has overflowed.

Qhull could not construct a clearly convex simplex from points:
- p0(v5): 5.1e+02 5.1e+02     0 4.6e+02
- p1144(v4):    75     2     0     5
- p7124(v3):     5     5     0     0
- p701(v2): 5.1e+02    26     0 2.3e+02
- p665(v1):     3 5e+02     0 2.3e+02


In [99]:
fig = scipy.spatial.voronoi_plot_2d(vor)
plt.gca().imshow(lr_dset.T)

In [180]:
def circumsphere_center(vertex_pos, tol=1e-3):
    """
    Function finds the center and the radius of the circumsphere of every tetrahedron.
    Reference:
    Fiedler, Miroslav. Matrices and graphs in geometry. No. 139. Cambridge University Press, 2011.
    (p.29 bottom: example 2.1.11)
    Code (slightly modified) from https://github.com/spatala/gbpy

    Parameters
    -----------------
    vertex_pos : numpy array
        The position of vertices of a tetrahedron
    tol : float
        Tolerance defined  to identify co-planar tetrahedrons
    Returns
    ----------
    circum_center : numpy array
        The center of the circumsphere
    circum_radius : float
        The radius of the circumsphere
    """
    
    if vertex_pos.shape[1] < 3:
        ax = vertex_pos[0, 0]
        ay = vertex_pos[0, 1]
        bx = vertex_pos[1, 0]
        by = vertex_pos[1, 1]
        cx = vertex_pos[2, 0]
        cy = vertex_pos[2, 1]
        d = 2 * (ax * (by - cy) + bx * (cy - ay) + cx * (ay - by))
        ux = ((ax * ax + ay * ay) * (by - cy) + (bx * bx + by * by) * (cy - ay) + (cx * cx + cy * cy) * (ay - by)) / d
        uy = ((ax * ax + ay * ay) * (cx - bx) + (bx * bx + by * by) * (ax - cx) + (cx * cx + cy * cy) * (bx - ax)) / d

        circum_center =np.array([ux, uy]) 
        circum_radius = np.linalg.norm(circum_center-vertex_pos[0])
        
        return np.array(circum_center), circum_radius
    dis_ij = scipy.spatial.distance.pdist(np.array(vertex_pos), 'euclidean')
    sq_12, sq_13, sq_14, sq_23, sq_24, sq_34 = np.power(dis_ij, 2)

    matrix_c = np.array([[0, 1, 1, 1, 1], [1, 0, sq_12, sq_13, sq_14], [1, sq_12, 0, sq_23, sq_24],
                         [1, sq_13, sq_23, 0, sq_34], [1, sq_14, sq_24, sq_34, 0]])

    det_matrix_c = (np.linalg.det(matrix_c))

    if det_matrix_c < tol:
        return np.array([0, 0, 0]), 0
    else:
        matrix = -2 * np.linalg.inv(matrix_c)
        circum_center = (matrix[0, 1] * vertex_pos[0, :] + matrix[0, 2] * vertex_pos[1, :] +
                         matrix[0, 3] * vertex_pos[2, :] +
                         matrix[0, 4] * vertex_pos[3, :]) / (matrix[0, 1] + matrix[0, 2] + matrix[0, 3] + matrix[0, 4])
        circum_radius = np.sqrt(matrix[0, 0]) / 2

    return np.array(circum_center), circum_radius

def voronoi_volumes(points):
    """
    Volumes of voronoi  cells from
    https://stackoverflow.com/questions/19634993/volume-of-voronoi-cell-python


    """
    v = scipy.spatial.Voronoi(points)
    vol = np.zeros(v.npoints)
    for i, reg_num in enumerate(v.point_region):
        indices = v.regions[reg_num]
        if -1 in indices: # some regions can be opened
            vol[i] = np.inf
        else:
            try:
                hull = scipy.spatial.ConvexHull(v.vertices[indices])
                vol[i] = hull.volume
            except:
                vol[i] = 0.
    return vol


def get_voronoi(tetrahedra, atoms, r_a, extent):
    """
    Find Voronoi vertices and keep track of associated tetrahedrons and interstitial radii
    
    Used in find_polyhedra function
    
    Parameters
    ----------
    tetrahedra: scipy.spatial.Delaunay object
        Delaunay tesselation
    atoms: ase.Atoms object
        the structural information
    r_a: float
        the atomic radius

    Returns
    -------
    voronoi_vertices: list
        list of positions of voronoi vertices
    voronoi_tetrahedra:
        list of indices of associated vertices of tetrahedra
    r_vv: list of float
        list of all interstitial sizes
    """

    voronoi_vertices = []
    voronoi_tetrahedrons = []
    r_vv = []
    for vertices in tetrahedra.vertices:
        voronoi, radius = circumsphere_center(atoms[vertices])
        
        if (voronoi >= 0).all() and (extent - voronoi > 0).all() and radius > 0.01:
            voronoi_vertices.append(voronoi)
            voronoi_tetrahedrons.append(vertices)
            r_vv.append(radius - r_a)
    return voronoi_vertices, voronoi_tetrahedrons, r_vv


def find_overlapping_spheres(voronoi_vertices, r_vv, r_a, cheat =1.):
    """Find overlapping spheres"""
    
    vertex_tree = scipy.spatial.cKDTree(np.array(voronoi_vertices)[:,:2])

    pairs = vertex_tree.query_pairs(r=r_a * 2)

    overlapping_pairs = []
    for (i, j) in pairs:
        if np.linalg.norm(voronoi_vertices[i] - voronoi_vertices[j]) < (r_vv[i] + r_vv[j])*cheat:
            overlapping_pairs.append([i, j])

    return np.array(sorted(overlapping_pairs))


def find_clusters(overlapping_pairs):
    """Make cluste
    We are using a breadth first to go through the list of overlapping spheres to determine clusters
    """
    visited_all = []
    clusters = []
    for initial in overlapping_pairs[:, 0]:
        if initial not in visited_all:
            # breadth first search
            visited = []  # the atoms we visited
            queue = [initial]
            while queue:
                node = queue.pop(0)
                if node not in visited_all:
                    visited.append(node)
                    visited_all.append(node)
                    # neighbors = overlapping_pairs[overlapping_pairs[:,0]==node,1]
                    neighbors = np.append(overlapping_pairs[overlapping_pairs[:, 1] == node, 0],
                                          overlapping_pairs[overlapping_pairs[:, 0] == node, 1])

                    for i, neighbour in enumerate(neighbors):
                        if neighbour not in visited:
                            queue.append(neighbour)
            clusters.append(visited)
    return clusters, visited_all


def make_polyhedrons(atoms, voronoi_vertices, voronoi_tetrahedrons, clusters, visited_all):
    """collect output data  and make dictionary"""

    polyhedra = {}
    for index, cluster in enumerate(clusters):
        cc = []
        for c in cluster:
            cc = cc + list(voronoi_tetrahedrons[c])
        hull = scipy.spatial.ConvexHull(atoms[list(set(cc))])
        faces = []
        triangles = []
        for s in hull.simplices:
            faces.append(atoms.positions[list(set(cc))][s])
            triangles.append(list(s))
        polyhedra[index] = {'vertices': atoms.positions[list(set(cc))], 'indices': list(set(cc)),
                            'faces': faces, 'triangles': triangles,
                            'length': len(list(set(cc))),
                            'combined_vertices': cluster,
                            'interstitial_index': index,
                            'interstitial_site': np.array(voronoi_tetrahedrons)[cluster].mean(axis=0),
                            'atomic_numbers': atoms.get_atomic_numbers()[list(set(cc))],
                            'volume': hull.volume}
        # 'coplanar': hull.coplanar}

    running_number = index + 0
    for index in range(len(voronoi_vertices)):
        if index not in visited_all:
            vertices = voronoi_tetrahedrons[index]
            hull = scipy.spatial.ConvexHull(atoms.positions[vertices])
            faces = []
            triangles = []
            for s in hull.simplices:
                faces.append(atoms.positions[vertices][s])
                triangles.append(list(s))

            polyhedra[running_number] = {'vertices': atoms.positions[vertices], 'indices': vertices,
                                         'faces': faces, 'triangles': triangles,
                                         'length': len(vertices),
                                         'combined_vertices': index,
                                         'interstitial_index': running_number,
                                         'interstitial_site': np.array(voronoi_tetrahedrons)[index],
                                         'atomic_numbers': atoms.get_atomic_numbers()[vertices],
                                         'volume': hull.volume}

            running_number += 1

    return polyhedra

##################################################################
# polyhedra functions
##################################################################


def find_polyhedra(atoms, r_a, extent, cheat=1.0):
    """ get polyhedra information from an ase.Atoms object

    This is following the method of Banadaki and Patala
    http://dx.doi.org/10.1038/s41524-017-0016-0

    Parameter
    ---------
    atoms: ase.Atoms object
        the structural information
    r_a: float
        the atomic radius

    Returns
    -------
    polyhedra: dict
        dictionary with all information of polyhedra
    """
    
    if not isinstance(r_a, (int, float)):
        raise TypeError('Atomic radius must be a real number')

    if not (0.5 < r_a < 2):
        print('Strange atomic radius, are you sure you know what you are doing?')
    print(atoms)
    tetrahedra = scipy.spatial.Delaunay(atoms)

    voronoi_vertices, voronoi_tetrahedrons, r_vv = get_voronoi(tetrahedra, atoms, r_a, extent)

    print(voronoi_vertices)
    overlapping_pairs = find_overlapping_spheres(voronoi_vertices, r_vv, r_a, cheat=cheat)

    clusters, visited_all = find_clusters(overlapping_pairs)

    polyhedra = make_polyhedrons(atoms, voronoi_vertices, voronoi_tetrahedrons, clusters, visited_all)

    return polyhedra


def sort_polyhedra_by_vertices(polyhedra, visible=range(4, 100), z_lim=[0, 100], verbose=False):
    indices = []

    for key, polyhedron in polyhedra.items():
        if 'length' not in polyhedron:
            polyhedron['length'] = len(polyhedron['vertices'])

        if polyhedron['length'] in visible:
            center = polyhedron['vertices'].mean(axis=0)
            if z_lim[0] < center[2] < z_lim[1]:
                indices.append(key)
                if verbose:
                    print(key, polyhedron['length'], center)
    return indices


# color_scheme = ['lightyellow', 'silver', 'rosybrown', 'lightsteelblue', 'orange', 'cyan', 'blue', 'magenta',
#                'firebrick', 'forestgreen']

##################################################################
# plotting functions
##################################################################


def plot_super_cell(super_cell, shift_x=0):
    """ make a super_cell to plot with extra atoms at periodic boundaries"""

    if not isinstance(super_cell, ase.Atoms):
        raise TypeError('Need an ase Atoms object')

    plot_boundary = super_cell * (2, 2, 3)
    plot_boundary.positions[:, 0] = plot_boundary.positions[:, 0] - super_cell.cell[0, 0] * shift_x

    del plot_boundary[plot_boundary.positions[:, 2] > super_cell.cell[2, 2] * 1.5 + 0.1]
    del plot_boundary[plot_boundary.positions[:, 1] > super_cell.cell[1, 1] + 0.1]
    del plot_boundary[plot_boundary.positions[:, 0] > super_cell.cell[0, 0] + 0.1]
    del plot_boundary[plot_boundary.positions[:, 0] < -0.1]
    plot_boundary.cell = super_cell.cell * (1, 1, 1.5)

    return plot_boundary


def plot_polyhedron(polyhedra, indices, center=False):
    if isinstance(indices, int):
        indices = [indices]
    if len(indices) == 0:
        print('Did not find any polyhedra')
        return {}

    center_point = np.mean(polyhedra[indices[0]]['vertices'], axis=0)

    if center:
        print(center_point)
        center = center_point
    else:
        center = [0, 0, 0]

    data = []
    for index in indices:
        polyhedron = polyhedra[index]

        vertices = polyhedron['vertices'] - center
        faces = np.array(polyhedron['triangles'])
        x, y, z = vertices.T
        i_i, j_j, k_k = faces.T

        mesh = dict(type='mesh3d',
                    x=x,
                    y=y,
                    z=z,
                    i=i_i,
                    j=j_j,
                    k=k_k,
                    name='',
                    opacity=0.2,
                    color=px.colors.qualitative.Light24[len(vertices) % 24]
                    )
        tri_vertices = vertices[faces]
        x_e = []
        y_e = []
        z_e = []
        for t_v in tri_vertices:
            x_e += [t_v[k % 3][0] for k in range(4)] + [None]
            y_e += [t_v[k % 3][1] for k in range(4)] + [None]
            z_e += [t_v[k % 3][2] for k in range(4)] + [None]

        # define the lines to be plotted
        lines = dict(type='scatter3d',
                     x=x_e,
                     y=y_e,
                     z=z_e,
                     mode='lines',
                     name='',
                     line=dict(color='rgb(70,70,70)', width=1.5))
        data.append(mesh)
        data.append(lines)
    return data


In [181]:
polyhedra  = find_polyhedra(atoms[:,:2], 4, extent=[lr_dset.shape[0],lr_dset.shape[1]])

Strange atomic radius, are you sure you know what you are doing?
[[509. 509.]
 [498. 509.]
 [461. 509.]
 ...
 [439. 475.]
 [237. 426.]
 [483. 467.]]
[array([5.  , 9.25]), array([8.78571429, 5.        ]), array([7.76923077, 8.55769231]), array([506.5, 280.5]), array([499.5, 272.5]), array([500. , 279.5]), array([505.5, 279.5]), array([500., 273.]), array([506.5, 273.5]), array([506., 273.]), array([505.58333333, 286.        ]), array([448.5, 233.5]), array([507.5,   5.5]), array([507.,   5.]), array([  5.5, 506.1]), array([348.5, 492.5]), array([  5.9, 403.5]), array([  6.58333333, 351.        ]), array([10.        , 10.41666667]), array([378.14285714,  32.        ]), array([378. ,   6.5]), array([ 43.41666667, 196.        ]), array([ 36.5, 196.5]), array([448.07142857, 175.5       ]), array([506.5, 190. ]), array([448., 233.]), array([428.92857143, 213.5       ]), array([507.1, 183.5]), array([505.9, 189.5]), array([288. ,   5.5]), array([502.,   5.]), array([507.,  94.]), array([256. 

AttributeError: 'numpy.ndarray' object has no attribute 'positions'

In [54]:
def make_new_vertices(vertices, extent, smallest_lattice_parameter):
    """ Determine whether vertices are too close and have to be replaced by median

    Part of get_graph function
    Parameters
    ----------
    vertices: numpy array (nx2)
        vertices of Voronoi tiles to be evaluated
    extent: list of float (4x1)
        extent of image
    smallest_lattice_parameter: float
        determines how far the Voronoi vertices have to be apart to be considered caused by distortion

    Returns
    -------
    new_voronoi: numpy array
        vertices of new Voronoi tiling
    """

    vertices_tree = scipy.spatial.cKDTree(vertices)

    dis = vertices_tree.query_ball_point(vertices, r=smallest_lattice_parameter * .7, p=2)  # , return_length=True)
    nn = vertices_tree.query_ball_point(vertices, r=smallest_lattice_parameter * .7, p=2, return_length=True)

    # handle nn > 2 differently Gerd

    new_voronoi = []
    for near in dis:
        if len(near) > 1:
            new = np.average(vertices[near], axis=0)
        elif len(near) > 0:
            new = vertices[near][0]
        else:
            new = [-1, -1]

        if (new > 0).all() and (new[0] < extent[1]) and (new[1] < extent[2]):
            new_voronoi.append(new)

    ver_sort = np.argsort(nn)
    nn_now = nn[ver_sort[-1]]
    done_list = []
    i = 1
    while nn_now > 2:
        close_vertices = dis[ver_sort[-i]]
        new_vert = []

        for vert in close_vertices:
            if vert not in done_list:
                new_vert.append(vert)

        done_list.extend(new_vert)
        # check whether necessary big_vertex = np.average(vertices[new_vert], axis=0)
        if len(new_vert) > 1:
            big_vertex = np.average(vertices[new_vert], axis=0)
            if (big_vertex[0] > 0) and (big_vertex[1] > 0):
                new_voronoi.append([big_vertex[0], big_vertex[1]])
        elif len(new_vert) > 0:
            new_voronoi.append([(vertices[new_vert[0]])[0], (vertices[new_vert[0]])[1]])
        i += 1
        nn_now = nn[ver_sort[-i]]

    # print(len(new_voronoi))
    new_voronoi = np.unique(new_voronoi, axis=0)
    return new_voronoi